### 데이터 타입 기록###
A. 데이터 설명

본 대회에서는 한국수력원자력(주)에서 제공받은 원자력 발전소 모사 데이터 및 실제 데이터가 제공됩니다. 

주어진 10분 사이에 발전소의 상태가 변하며 변하기 전 발전소 상태를 상태_A, 변한 후 상태를 상태_B라고 할 때 

데이터를 기반으로 상태_B를 판단하는 것이 대회의 목적입니다. 모든 데이터는 같은 상태 (상태_A)에서 시작합니다. 



모사 데이터는 0초에서 15초 사이에 상태가 변하기 시작합니다. 모든 데이터가 숫자로 이루어져 있습니다. 

반면 실제 데이터는 10초에 상태가 변하기 시작합니다. 실제 데이터에는 Bad, CLOSE, Equip Fail, No Data, Normal, OFF, ON, OPEN, System.Char[] 등 문자열 데이터를 포함하고 있으며 결측치가 있을 수 있습니다. 



훈련 데이터로는 10분 데이터가 주어지며 테스트 데이터로는 1분 간 데이터가 주어집니다. 즉 1분 데이터만 사용하여 평가합니다. 



B. 파일 설명

1. train.zip(9.81GB) : 훈련 데이터가 포함된 zip 파일입니다.

2. test.zip(879MB) : 테스트 데이터가 포함된 zip 파일입니다.

   train.zip, test.zip 내 파일이름은 해당되는 고유한 번호 (id) 로 되어 있습니다. train과 test의 id는 중복되지 않습니다.

3. train_label.csv(6.78KB) : 훈련데이터의 id (파일 이름)에 해당되는 상태_B를 정리한 파일입니다. 

4. sample_submission.csv(283KB) : 제출 형식과 동일한 샘플 데이터 입니다.

5. additional_data.zip(13.9MB) : 실제 발전소 운전 데이터 입니다. 참고자료로 주어지는 파일입니다. (컬럼은 train 및 test 데이터와 같습니다.)



데이터 파일 컬럼에 대한 설명은 아래와 같습니다.

time : 발전소 운전 경과 시간 입니다. 단위는 초 입니다.

V0000~V5120 : 발전소 운영 중 측정되는 변수 입니다. 비식별화 되어 제공됩니다.
if) column 이 많을경우 로드한 후 train.info(), test.info() 를 사용해서 int, float, object 구별한다

Anonymized 된 binary classifiacation  
258개의 column  
262000 개의 row  


O 코드 관련

1) 입상자는 코드 제출 필수. 제출 코드는 예측 결과를 리더보드 점수로 복원할 수 있어야 함

2) 코드 제출시 확장자가 R user는 R or .rmd. Python user는 .py or .ipynb

3) 코드에 ‘/data’ 데이터 입/출력 경로 포함 제출

4) 전체 프로세스를 일목요연하게 정리하여 주석을 포함하여 하나의 파일로 제출

5) 모든 코드는 오류 없이 실행되어야 함(라이브러리 로딩 코드 포함되어야 함).

6) 코드와 주석의 인코딩은 모두 UTF-8을 사용하여야 함 

## metric
평가 지표는 Log Loss를 사용합니다.

In [1]:
import numpy as np
import pandas as pd

from utils import load_eda_statistic_pd,reduce_mem_usage

import os
import gc

import matplotlib.pyplot as plt
import seaborn as sns

import dill as pickle

from catboost import Pool

In [17]:
train = pd.read_csv("data/train/0.csv")
test = pd.read_csv("data/test/828.csv")
sample_submission = pd.read_csv("data/sample_submission.csv")
print("{} observations and {} features in {} train set.".format(train.shape[0],train.shape[1], len(os.listdir("data/train"))))
print("{} observations and {} features in {} test set.".format(test.shape[0],test.shape[1],len(os.listdir("data/test"))))
print("train : test row ratio {} and file {}".format(train.shape[0] / test.shape[0], len(os.listdir("data/train"))/len(os.listdir("data/test"))))

600 observations and 5122 features in 828 train set.
60 observations and 5122 features in 720 test set.
train : test row ratio 10.0 and file 1.15
